### Panda interactive Control

1. open a new terminal
1. launch a file by **roslaunch enable_panda.launch**

In [1]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
import rospy
from brl_panda.panda_ros_py import *
from brl_panda.spatial_math.spatial_math import *

### Plot config

In [3]:
base.axeslength = 0.1
base.axeswidth = 1
SE3().plot_init(dims=np.array([-1,0,-1,0,-0,1])*0.5)

In [4]:
SE3().plot_clear()

In [4]:
# This makes this notebook script as a node
rospy.init_node("panda_interactive")

In [5]:
g = Gripper()
a = Arm()

[INFO] [1633528096.612290]: Waiting for gripper action servers... 
[INFO] [1633528097.566933]: Gripper action servers found!
[INFO] [1633528097.572045]: Arm is connected


In [6]:
def generate_trajectory(n, start_pose, end_pose):
    R_start, p_start = start_pose.R, start_pose.p
    R_end, p_end = end_pose.R, end_pose.p
    s_list = np.linspace(0,1,n)
    err_axis, err_angle = SO3(R_start.T@R_end).to_axisangle()
    
    T_list = []
    for s in s_list:
        R = R_start @ SO3().axisangle(err_axis, err_angle*s).R
        p = p_start + s * (p_end - p_start)
        T_list.append(SE3(R,p))
    return T_list

def move(arm, traj):
    for T in traj:
        ori, pos = T.to_qtn_trans()
        arm.set_eq_pose(pos, ori)
        #rospy.loginfo("moving... pos:{}, ori:{} \n".format(np.round(pos,2),np.round(ori,2)))
        rospy.sleep(0.1)

In [13]:
T = SE3(a.get_EE_pose())
T.plot(frame="EE")

In [8]:
T2 = T@SE3().trans([-0.1,0,0])
T2.plot(frame="target")

In [9]:
T_list = generate_trajectory(100, T, T2)

In [10]:
move(a, T_list)

In [12]:
a.set_translational_stiffness(300)

In [96]:
a.get_EE_pose()

array([[-0.96546899, -0.20492533, -0.16079953, -0.45094036],
       [-0.18999748,  0.9763097 , -0.103447  , -0.34976985],
       [ 0.17818905, -0.06932336, -0.98155093,  0.16045833],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])